In [1]:
"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding more
average features and weekly average features on it.
"""
from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb

In [2]:
order=pd.read_csv('jdata_user_order.csv',parse_dates=['o_date'])
order.head()

,user_id,sku_id,o_id,o_date,o_area,o_sku_num
0,1,80036,480066,2017-03-09,14,1
1,1,16607,106933,2017-01-12,14,1
2,1,80036,542604,2017-04-14,14,1
3,1,80036,196335,2017-04-30,14,1
4,1,80036,42502,2017-03-22,14,2


In [3]:
user=pd.read_csv('jdata_user_basic_info.csv').set_index('user_id')
user.head()

,age,sex,user_lv_cd
user_id,,,
1,3,2,3
2,3,2,3
3,3,0,5
4,3,2,3
5,4,2,2


In [4]:
sku=pd.read_csv('jdata_sku_basic_info.csv',usecols=[0,1,2,3]).set_index('sku_id')
sku.head()

,price,cate,para_1
sku_id,,,
1,364.2,71,247.95
2,36.7,46,6.40
3,953.3,71,178.92
4,110.6,83,6366.88
5,241.5,71,31.74


In [5]:
le = LabelEncoder()
sku['cate'] = le.fit_transform(sku['cate'].values)

user['sex'] = le.fit_transform(user['sex'].values)
user['user_lv_cd'] = le.fit_transform(user['user_lv_cd'].values)

In [6]:
sku.cate.value_counts() #cate 1 5

3    41959
5    30344
4     7638
1     7380
0     7333
2     4758
Name: cate, dtype: int64

In [7]:
action=pd.read_csv('jdata_user_action.csv',parse_dates=['a_date'])
action1=action[action.a_type==1] #liulan
action2=action[action.a_type==2] #guanzhu
action1.drop('a_type',axis=1,inplace=True)
action2.drop('a_type',axis=1,inplace=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [10]:
del action

In [11]:
order=order.ix[:,[0,1,3,5]]
user_item_key=order.drop_duplicates(['user_id','sku_id'])[['user_id','sku_id']]
print(user_item_key,user_item_key.shape)
order=order.groupby(by=['user_id','sku_id','o_date']).sum()
order=order.unstack(-1).fillna(0)
order.columns=order.columns.get_level_values(1)
order.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


        user_id  sku_id
0             1   80036
1             1   16607
6             2   44666
7             3   71817
8             3   18217
9             3    2816
11            3   82091
12            3   91286
13            3   28001
15            3   38635
16            3   80409
17            3   35919
19            3   16586
21            3    9833
23            3   16080
25            4   75542
26            5   11071
27            6   69985
28            6   24052
30            7   93883
31            7   65997
34            7    7169
36            7   18409
37            7   45817
40            8   33236
41            8   76072
43            9   93883
44            9   44554
45            9   44542
46            9   78567
...         ...     ...
792679    98919   87667
792680    98919   11424
792681    98920   20656
792682    98921   81182
792684    98922   54477
792685    98922    1132
792686    98922   91630
792687    98922   43172
792688    98922   78997
792689    98922 

o_date          2016-05-01  2016-05-02  2016-05-03  2016-05-04  2016-05-05  \
user_id sku_id                                                               
1       16607          0.0         0.0         0.0         0.0         0.0   
        80036          0.0         0.0         0.0         0.0         0.0   
2       44666          0.0         0.0         0.0         0.0         0.0   
3       2816           0.0         0.0         0.0         0.0         0.0   
        9833           0.0         0.0         0.0         0.0         0.0   

o_date          2016-05-06  2016-05-07  2016-05-08  2016-05-09  2016-05-10  \
user_id sku_id                                                               
1       16607          0.0         0.0         0.0         0.0         0.0   
        80036          0.0         0.0         0.0         0.0         0.0   
2       44666          0.0         0.0         0.0         0.0         0.0   
3       2816           0.0         0.0         0.0         0.0         0.0   
        9833           0.0         0.0         0.0         0.0         0.0   

o_date             ...      2017-04-21  2017-04-22  2017-04-23  2017-04-24  \
user_id sku_id     ...                                                       
1       16607      ...             0.0         0.0         0.0         0.0   
        80036      ...             0.0         0.0         0.0         0.0   
2       44666      ...             0.0         0.0         0.0         2.0   
3       2816       ...             0.0         0.0         0.0         0.0   
        9833       ...             0.0         0.0         0.0         0.0   

o_date          2017-04-25  2017-04-26  2017-04-27  2017-04-28  2017-04-29  \
user_id sku_id                                                               
1       16607          0.0         0.0         0.0         0.0         0.0   
        80036          0.0         0.0         0.0         0.0         0.0   
2       44666          0.0         0.0         0.0         0.0         0.0   
3       2816           0.0         0.0         0.0         0.0         0.0   
        9833           0.0         0.0         0.0         0.0         0.0   

o_date          2017-04-30  
user_id sku_id              
1       16607          0.0  
        80036          1.0  
2       44666          0.0  
3       2816           0.0  
        9833           0.0  

[5 rows x 365 columns]

In [12]:
order = order[
    pd.date_range("2017-01-01", periods=89+31)].copy()

In [13]:
order.head()

o_date          2017-01-01  2017-01-02  2017-01-03  2017-01-04  2017-01-05  \
user_id sku_id                                                               
1       16607          0.0         0.0         0.0         0.0         0.0   
        80036          0.0         0.0         0.0         0.0         0.0   
2       44666          0.0         0.0         0.0         0.0         0.0   
3       2816           0.0         0.0         0.0         0.0         0.0   
        9833           0.0         0.0         0.0         0.0         0.0   

o_date          2017-01-06  2017-01-07  2017-01-08  2017-01-09  2017-01-10  \
user_id sku_id                                                               
1       16607          0.0         0.0         0.0         0.0         0.0   
        80036          0.0         0.0         0.0         0.0         0.0   
2       44666          0.0         0.0         0.0         0.0         0.0   
3       2816           0.0         0.0         0.0         0.0         0.0   
        9833           0.0         0.0         0.0         0.0         0.0   

o_date             ...      2017-04-21  2017-04-22  2017-04-23  2017-04-24  \
user_id sku_id     ...                                                       
1       16607      ...             0.0         0.0         0.0         0.0   
        80036      ...             0.0         0.0         0.0         0.0   
2       44666      ...             0.0         0.0         0.0         2.0   
3       2816       ...             0.0         0.0         0.0         0.0   
        9833       ...             0.0         0.0         0.0         0.0   

o_date          2017-04-25  2017-04-26  2017-04-27  2017-04-28  2017-04-29  \
user_id sku_id                                                               
1       16607          0.0         0.0         0.0         0.0         0.0   
        80036          0.0         0.0         0.0         0.0         0.0   
2       44666          0.0         0.0         0.0         0.0         0.0   
3       2816           0.0         0.0         0.0         0.0         0.0   
        9833           0.0         0.0         0.0         0.0         0.0   

o_date          2017-04-30  
user_id sku_id              
1       16607          0.0  
        80036          1.0  
2       44666          0.0  
3       2816           0.0  
        9833           0.0  

[5 rows x 120 columns]

In [14]:
sku=sku.reindex(order.index.get_level_values(1))
user=user.reindex(order.index.get_level_values(0))
sku.shape,user.shape,order.shape

((497041, 3), (497041, 3), (497041, 120))

In [15]:
action1=user_item_key.merge(action1,on=['user_id','sku_id'],how='left')
action1=action1.groupby(by=['user_id','sku_id','a_date']).sum()
action1=action1.unstack(-1).fillna(0)
action1.columns=action1.columns.get_level_values(1)
action1.head()

a_date          2016-05-01  2016-05-02  2016-05-03  2016-05-04  2016-05-05  \
user_id sku_id                                                               
1       16607          0.0         0.0         0.0         0.0         0.0   
        80036          0.0         0.0         0.0         0.0         0.0   
2       44666          0.0         0.0         0.0         0.0         0.0   
3       2816           0.0         0.0         0.0         0.0         0.0   
        9833           0.0         0.0         0.0         0.0         0.0   

a_date          2016-05-06  2016-05-07  2016-05-08  2016-05-09  2016-05-10  \
user_id sku_id                                                               
1       16607          0.0         0.0         0.0         0.0         0.0   
        80036          0.0         0.0         0.0         0.0         0.0   
2       44666          0.0         0.0         0.0         0.0         0.0   
3       2816           0.0         0.0         0.0         0.0         0.0   
        9833           0.0         0.0         0.0         0.0         0.0   

a_date             ...      2017-04-21  2017-04-22  2017-04-23  2017-04-24  \
user_id sku_id     ...                                                       
1       16607      ...             0.0         0.0         0.0         0.0   
        80036      ...             0.0         0.0         0.0         0.0   
2       44666      ...             0.0         0.0         0.0         2.0   
3       2816       ...             0.0         0.0         0.0         0.0   
        9833       ...             0.0         0.0         0.0         0.0   

a_date          2017-04-25  2017-04-26  2017-04-27  2017-04-28  2017-04-29  \
user_id sku_id                                                               
1       16607          0.0         0.0         0.0         0.0         0.0   
        80036          0.0         0.0         0.0         0.0         0.0   
2       44666          2.0         0.0         0.0         0.0         0.0   
3       2816           0.0         0.0         0.0         0.0         0.0   
        9833           0.0         0.0         0.0         0.0         0.0   

a_date          2017-04-30  
user_id sku_id              
1       16607          0.0  
        80036          6.0  
2       44666          0.0  
3       2816           0.0  
        9833           0.0  

[5 rows x 365 columns]

In [16]:
action1 = action1[
    pd.date_range("2017-01-01", periods=89+31)].copy()
action1.shape

(454257, 120)

In [17]:
action2=user_item_key.merge(action2,on=['user_id','sku_id'],how='left')
action2=action2.groupby(by=['user_id','sku_id','a_date']).sum()
action2=action2.unstack(-1).fillna(0)
action2.columns=action2.columns.get_level_values(1)
action2 = action2[
    pd.date_range("2017-01-01", periods=89+30)].copy()###少一天
action2.shape

(28300, 119)

In [20]:
order_item = order.groupby('sku_id')[order.columns].sum()
action1_item = action1.groupby('sku_id')[action1.columns].sum()
action2_item = action2.groupby('sku_id')[action2.columns].sum()

In [21]:
#user x cate
# df_2017_store_class = df_2017.reset_index()
# df_2017_store_class['class'] = items['class'].values
# df_2017_store_class_index = df_2017_store_class[['class', 'store_nbr']]
# df_2017_store_class = df_2017_store_class.groupby(['class', 'store_nbr'])[df_2017.columns].sum()

# df_2017_promo_store_class = promo_2017.reset_index()
# df_2017_promo_store_class['class'] = items['class'].values
# df_2017_promo_store_class_index = df_2017_promo_store_class[['class', 'store_nbr']]
# df_2017_promo_store_class = df_2017_promo_store_class.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()

In [22]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

def prepare_dataset(df, promo_df, t2017, is_train=True, name_prefix=None):
    X = {
        "action1_3_2017": get_timespan(action1, t2017, 3, 3).sum(axis=1).values,
        "action1_7_2017": get_timespan(action1, t2017, 7, 7).sum(axis=1).values,
        "action1_14_2017": get_timespan(action1, t2017, 14, 14).sum(axis=1).values,
        "action1_29_2017": get_timespan(action1, t2017, 29, 29).sum(axis=1).values,
    }

#     for i in [1, 2, 3, 7, 14, 29]:
#         tmp1 = get_timespan(df, t2017, i, i)
#         tmp2 = (get_timespan(promo_df, t2017, i, i) > 0) * 1

#         X['has_promo_mean_%s' % i] = (tmp1 * tmp2.replace(0, np.nan)).mean(axis=1).values
#         X['has_promo_mean_%s_decay' % i] = (tmp1 * tmp2.replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values

#         X['no_promo_mean_%s' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan)).mean(axis=1).values
#         X['no_promo_mean_%s_decay' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values

    for i in [1, 2, 3, 7, 14, 29]:
        tmp = get_timespan(df, t2017, i, i)
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s_decay' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s' % i] = tmp.mean(axis=1).values
        X['median_%s' % i] = tmp.median(axis=1).values
        X['min_%s' % i] = tmp.min(axis=1).values
        X['max_%s' % i] = tmp.max(axis=1).values
        X['std_%s' % i] = tmp.std(axis=1).values

#     for i in [1, 2, 3, 7, 14, 29]:
#         tmp = get_timespan(df, t2017 + timedelta(days=-7), i, i)
#         X['diff_%s_mean_2' % i] = tmp.diff(axis=1).mean(axis=1).values
#         X['mean_%s_decay_2' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
#         X['mean_%s_2' % i] = tmp.mean(axis=1).values
#         X['median_%s_2' % i] = tmp.median(axis=1).values
#         X['min_%s_2' % i] = tmp.min(axis=1).values
#         X['max_%s_2' % i] = tmp.max(axis=1).values
#         X['std_%s_2' % i] = tmp.std(axis=1).values

#     for i in [7, 14, 30, 60, 140]:
#         tmp = get_timespan(df, t2017, i, i)
#         X['has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
#         X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
#         X['first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

#         tmp = get_timespan(promo_df, t2017, i, i)
#         X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
#         X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
#         X['first_has_promo_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

#     tmp = get_timespan(promo_df, t2017 + timedelta(days=16), 15, 15)
#     X['has_promo_days_in_after_15_days'] = (tmp > 0).sum(axis=1).values
#     X['last_has_promo_day_in_after_15_days'] = i - ((tmp > 0) * np.arange(15)).max(axis=1).values
#     X['first_has_promo_day_in_after_15_days'] = ((tmp > 0) * np.arange(15, 0, -1)).max(axis=1).values

    for i in range(1, 16):
        X['day_%s_2017' % i] = get_timespan(df, t2017, i, 1).values.ravel()

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, t2017, 28-i, 4, freq='7D').mean(axis=1).values
#         X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, t2017, 140-i, 20, freq='7D').mean(axis=1).values

#     for i in range(-16, 16):
#         X["promo_{}".format(i)] = promo_df[t2017 + timedelta(days=i)].values.astype(np.uint8)

    X = pd.DataFrame(X)

    if is_train:
        y = result[
            pd.date_range(t2017, periods=30)
        ]
        y=y.sum(axis=1).values
        return X, y
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    return X


In [24]:
print("Preparing dataset...")
t2017 = date(2017, 2, 1)
num_days = 3
X_l, y_l = [], []
for i in range(num_days):
    delta = timedelta(days=29 * i)
    X_tmp, y_tmp = prepare_dataset(order, action1, t2017 + delta)

    X_tmp2 = prepare_dataset(order_item, action1_item, t2017 + delta, is_train=False, name_prefix='item')
    X_tmp2.index = action1_item.index
    X_tmp2 = X_tmp2.reindex(action1.index.get_level_values(1)).reset_index(drop=True)
    
    X_tmp3 = prepare_dataset(order_item, action2_item, t2017 + delta, is_train=False, name_prefix='item')
    X_tmp3.index = action2_item.index
    X_tmp3 = X_tmp2.reindex(action2.index.get_level_values(1)).reset_index(drop=True)
    
#     X_tmp3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, t2017 + delta, is_train=False, name_prefix='store_class')
#     X_tmp3.index = df_2017_store_class.index
#     X_tmp3 = X_tmp3.reindex(df_2017_store_class_index).reset_index(drop=True)

    X_tmp = pd.concat([X_tmp, X_tmp2, X_tmp3, items.reset_index(), stores.reset_index()], axis=1)
    X_l.append(X_tmp)
    y_l.append(y_tmp)

    del X_tmp2
    gc.collect()

X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

Preparing dataset...


ValueError: arrays must all be same length